In [1]:
import time
import random

# Set seed.
random.seed(a=100)

# Create our default list.
short_list = list(random.sample(range(1000000), 10))
long_list = list(random.sample(range(1000000), 10000))

Now that we've covered some basic data structures, let's get into the things we can _do_ with those structures a little more. Sure, we've covered some basic operations like adding and deleting elements, but what about more complex processes?

These processes are called algorithms. (Ok, they're actually all technically within the class of algorithms, but these are algorithms we really think of as algorithms, rather than just single steps).

Algorithms lend themselves to a lot of comparisons to try to illustrate what they are. Some people call them recipes. Others call them directions. The key is this: it's the set of steps necessary for a computer to accomplish a specific task.

But what kind of task?

Really any kind of task you want. But the most common to discuss (and the most common to show up in interviews) is sorting. Now, there are many many kinds of sorting algorithms, a brief summary of which you can get from the [Sorting Algorithm wiki page](https://en.wikipedia.org/wiki/Sorting_algorithm).

We won't cover all of them here, but we'll talk about a few of them to draw out differences in performance, efficiency, and style.

As a note, above we're defining `short_list` and `long_list` which will be our default random lists. The short one will be used to validate that our algorithm works, and the long list to compare computation times across sorting strategies. Standardizing both will let us use them as the same baselines for each algorithm we explore. We're also only going to use lists here, but this does generalize more broadly.

## The Task of Sorting

Just for completeness, let's briefly review what we mean by sorting. The most common example used to talk about methods of sorting is a hand of cards. When you get a hand of cards in a game you want to organize them is some kind of reasonable fashion. This makes it easier to know what cards are in your hand, and to find and access the cards you want.

Now, there are many different ways to sort. Everyone kind of has their own method. Some will pick cards up one and a time and sequence them as they go. Others will move through their hand reorganizing card by card. You could even just randomly shuffle them again and again until they are sorted (though this would not be a very efficient approach). Different methods work best in different games and with different sized hands. These same general concepts apply to sorting lists.

All of our lists will have a set of values, in our case integers ranging from 0 to 1,000,000. Our goal is to return this list ordered from smallest to largest in the least amount of time. In situations where there are duplicates, those duplicates will maintain their original order (this preserves _stability_ in our algorithm). Our goal is to sort the list as efficiently as possible. This will be _measured_ in runtime, but also _discussed_ in terms of steps.

## Insertion Sort

One of the simplest methods of sorting is the _insertion sort_. In this case we maintain two lists. First is our original list. Then we have a new list, which will be ordered. To add elements to that list, we take an element from our original list and then move through our new list, stopping and inserting it in the appropriate place. We do this by placing it in the position ahead of the first element in the new list that is larger than our chosen element. If none are larger then it is added to the end.

This gives the nice property that our result will always be a sorted list which will grow to encompass the entire original set.

Let's write it up quickly.

In [3]:
def insert_sort(input_list):
    # Copy the input to a new list so we don't modify the original.
    new_list = input_list
    
    # Iterate through the list.
    for i in range(len(new_list)):
        # Assign place to a new variable.
        j = i
        
        # Move through the list as long as the previous position is larger
        # than the current element of list.
        while j > 0 and new_list[j - 1] > new_list[j]:
            
            # Swap places.
            new_list[j - 1], new_list[j] = new_list[j], new_list[j - 1]
            
            # Reduce j by one.
            j = j - 1
    return new_list

You can think about insertion sort as being like organizing a poker hand from left to right, where you scan your hand from left to right, picking up out of place cards when you see them and moving them to the left as far as they need to go. Here is an visualization from Wikipedia:

![Animated insertion sort visualization](https://upload.wikimedia.org/wikipedia/commons/0/0f/Insertion-sort-example-300px.gif)

If you want to play around with insertion sort or other algorithms visually, check out [VisuAlgo](https://visualgo.net/en/sorting). Feel free to escape out of the slides and tinker directly with the visualizations.

The Python function above implements this algorithm and makes each individual _step_ clear. Lets apply it to our short and long lists.

In [4]:
# Start the timer.
start_time = time.time()

# Run our insertion sort.
insert_sort(short_list)

# Print time to show runtime.
print("--- %s seconds ---" % (time.time() - start_time))
print(insert_sort(short_list))

--- 6.604194641113281e-05 seconds ---
[152745, 183236, 366725, 412125, 477025, 481850, 739784, 767514, 808225, 997948]


In [5]:
# Test on the long list.
start_time = time.time()

insert_sort(long_list)

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.174955129623413 seconds ---


So that seems to work! We've created a resonable insertion sort that works through the list piecewise and inserts each element in the appropriate spot.

However this also revealed something else important about insertion sort: it doesn't scale well. We noticed that it took 11 seconds on the long list. That's an unacceptably long amount of time. This is because its best case time is very different from its worst case.

If the list is already ordered this kind of sort takes n steps to complete. It simply iterates through the list. However, if your list is perfectly out of order it can take asymptotically n-squared steps (in big-O notation, $\mathcal{O}(n^2)$) as we have $n$ elements and our algorithm will potentially look through each element in the sorted list before inserting the element. This means computation can get more intensive quite quickly, as evidenced by the runtime of our long list. Think about what a square function looks like if you graph it. It grows at an ever increasing rate. The [wiki for Insertion Sort](https://en.wikipedia.org/wiki/Insertion_sort#Best.2C_worst.2C_and_average_cases) includes this lovely animation showing this method and the challenge that comes from it.

![Insertion sort](https://upload.wikimedia.org/wikipedia/commons/4/42/Insertion_sort.gif)

So, how could we do better?

## Merge Sort

One method for improving this is to use a _merge sort_. Merge sort takes advantage of fact that merging two small sorted lists into one large sorted list is fast. Merge sort starts by breaking our large list into single element sublists. It's a bit wonky, but if you think about it these single-element lists are each inherently ordered. Then we merge those single-element lists together into ordered pairs, reading from a single end to preserve their order. We repeat this process and arrive ultimately at a sorted list.

This will look much more complex in code but the concept is easier to understand with an example. Let's try it first with a very basic manual walkthrough.

If our list were `[3, 7, 2, 4]` The algorithm would first break it up into four pieces `[3], [7], [2], [4]`. Then we could split that into two groups and merge each by order, resulting in `[3, 7], [2, 4]`. Finally we bring those two lists together to get `[2, 3, 4, 7]`. It's more efficient because in any merge we only have to look at the leading entry from each prior list. For that final merge in the first step we're only comparing 3 to 2 and 4 to 7, since we already know 4 and 7 are larger than their prior entries.

Why does that give us an advantage? It's because we won't have to handle large unordered data. We always know what's next in any merge is from one of two places (the next element in each of our two lists we're merging). It's over really long lists that this advantage becomes decidedly worthwhile. This technique is called _divide and conquer_. Our insertion sort tries to solve the whole problem in one piece. Sometimes that's great. But in the case of sorting a long list, that process is inefficient. By breaking our task into smaller ones, in this case sorting small lists and then merging those ordered lists together, we make significant gains in efficiency. Here's a bigger example.

![Animated merge sort](https://upload.wikimedia.org/wikipedia/commons/c/cc/Merge-sort-example-300px.gif)

This tradeoff is a common feature of algorithms. It's often easiest to write something that tries to solve the problem all in one go. The logic is often easier to visualize. However, thinking about how to break one big problem into several smaller problems is a common way to gain efficiency.

Here's what the code for a merge sort looks like (with `merge` written as a separate function):

In [6]:
# Our merge function takes two ordered lists and merges them together into one ordered list

def merge(a, b):
    # Check for empty list.
    if len(a) == 0 or len(b) == 0:
        return a or b
    
    # Start with an empty result.
    result = []
    # Track two indexes.
    i, j = 0, 0
    # Set a while condition to ensure we iterate only for the length of our two lists.
    while (len(result) < len(a) + len(b)):
        # If a's next element is lower append that element to our result.
        if a[i] < b[j]:
            result.append(a[i])
            i += 1
        # Otherwise append b's next element.
        else:
            result.append(b[j])
            j += 1
        # When one list is empty just append everything from the other list and stop.
        if i == len(a) or j == len(b):
            result.extend(a[i:] or b[j:])
            break 

    return result

def merge_sort(lst):
    if len(lst) < 2:
        return lst

    mid = int(len(lst) / 2)
    a = merge_sort(lst[:mid])
    b = merge_sort(lst[mid:])

    return merge(a, b)


In [7]:
# Test on short list.
# Start timer.
start_time = time.time()

# Run our insertion sort.
merge_sort(short_list)

# Print time to show runtime.
print("--- %s seconds ---" % (time.time() - start_time))
print(insert_sort(short_list))

--- 8.296966552734375e-05 seconds ---
[152745, 183236, 366725, 412125, 477025, 481850, 739784, 767514, 808225, 997948]


In [8]:
# Test on long list.
start_time = time.time()

merge_sort(long_list)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04842400550842285 seconds ---


Now, this algorithm is implemented _recursively_, meaning the function nests within itself, running multiple times until a stopping condition is met. This is how we create multiple layers of lists to merge together. Recursion, again, is a common feature of algorithms. It's a way of nesting an algorithm within itself so that it keeps going until the problem is actually solved and you don't have to specify how many times something should run.

It is also much, much faster, a tenth of a second instead of 11 seconds, and less complex: $\mathcal{O}(n\log{}n)$ instead of $\mathcal{O}(n^2))$.

This break-down-and-merge method means that when combining shorter lists into longer lists, we can use the knowledge that the shorter lists are already sorted to cut down on the number of comparisons we need to make.  As a result, we don't have to potentially look through all other sorted elements in order to place a single element of our list. It no longer scales quadtratically, but in quasilinear time.

## The Default Sort Method

Now, all of this is fine and dandy, but it's not the only way to sort things.

We also have a simpler way. Kind of a cheating way. Python lists have a built in `.sort()` method and there's also the built-in `sorted()` function. Let's see how that performs.

In [9]:
# Start Timer
start_time = time.time()

# Sort the default list. Note that .sort() will sort in place, which would alter default_list.
sorted(long_list)

# Print time to show runtime
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00037479400634765625 seconds ---


Now that is much faster than either of ours, so for most cases it's worth just using the built ins. The reasons for this efficiency are many and partially have to do with the fact that it isn't actually written in Python, but Cython, which allows it to run in a version of C from Python that is much faster than generic Python.

So why are you learning slow ways to sort that take a lot of work to implement? It's worth understanding how these algorithms function at their most basic level and how we can work with them to build our own bespoke tools. The more complex algorithms you'll implement later will rely on these fundamentals.

## DRILL

Return to the [sorting wiki page](https://en.wikipedia.org/wiki/Sorting_algorithm) and pick an algorithm we haven't covered here (you probably want to pick one of the simpler ones, but it's up to you. Implement it in Python below and see how it compares in sorting our short and long lists. You should be able to easily find guides on how to implement any of the algorithms. Can you figure out why it runs faster or slower than our other sorting algorithms?

Some good sorts to try are:
* Heap Sort
* Selection Sort
* QuickSort

In [14]:
#SelectionSort:

import sys 
A = long_list 
  
# Traverse through all array elements 
for i in range(len(A)): 
      
    # Find the minimum element in remaining  
    # unsorted array 
    min_idx = i 
    for j in range(i+1, len(A)): 
        if A[min_idx] > A[j]: 
            min_idx = j 
              
    # Swap the found minimum element with  
    # the first element         
    A[i], A[min_idx] = A[min_idx], A[i] 
  
# Driver code to test above 
print ("Sorted array") 
for i in range(len(A)): 
    print("%d" %A[i]),  

Sorted array
29
174
316
373
424
430
705
762
786
830
1017
1456
1652
1767
2115
2123
2241
2259
2450
2459
2765
2775
2776
2807
2973
3152
3183
3356
3376
3595
3678
3863
3892
3975
4016
4064
4090
4360
4772
4786
5280
5355
5875
5916
5919
5956
6060
6385
6493
6523
6599
6874
6916
6955
7093
7331
7332
7537
7669
8164
8266
8312
8388
8557
8604
8717
8860
9034
9274
9314
9430
9458
9521
9553
9588
9597
9808
9823
9891
9911
9944
9954
9990
10174
10204
10213
10252
10426
10757
10770
10945
10977
11019
11031
11380
11476
11507
11510
11658
11694
11757
11794
11869
11875
12006
12170
12191
12210
12344
12415
12430
12688
12711
12746
12752
12768
12819
12885
13022
13116
13135
13389
13457
13680
13809
13818
13822
13897
14208
14314
14506
14615
14626
14671
14885
14928
15079
15152
15172
15195
15317
15387
15415
15681
15796
16008
16051
16118
16196
16267
16332
16386
16450
16706
16889
16979
17002
17068
17344
17514
17556
17606
17632
17863
18069
18131
18220
18406
18479
18650
18787
18912
19120
19307
19326
19393
19411
19539
19652
19734
1

192097
192177
192283
192541
192574
192597
192725
192764
192987
193114
193221
193247
193318
193412
193570
193697
193898
193933
194045
194106
194155
194362
194397
194732
194825
194976
194993
195023
195134
195328
195588
195642
195707
195929
196165
196434
196469
196482
196511
196574
196688
196872
196930
196948
196952
197072
197123
197241
197258
197712
197749
197756
197765
197780
197812
197884
197889
197962
198059
198122
198174
198347
198361
198426
198732
198769
198944
199152
199215
199222
199400
199478
199726
199731
199732
199959
200111
200204
200495
200506
200675
200705
200750
200760
200769
200947
200970
201165
201287
201301
201322
201328
201407
201548
201693
201998
202058
202135
202187
202195
202216
202273
202383
202385
202398
202506
202518
202791
202889
203037
203128
203209
203234
203235
203321
203335
203341
203379
203443
203513
203636
203667
203672
203710
203795
203823
203839
203867
203886
203889
203945
204020
204070
204183
204314
204317
204406
204594
204615
204877
205311
205449
205567

341400
341450
341514
341734
342141
342153
342195
342355
342366
342370
342630
342662
342759
342911
343056
343069
343079
343115
343167
343233
343266
343300
343415
343546
343556
343841
344141
344298
344307
344321
344639
344711
344739
344748
344986
345030
345031
345168
345179
345299
345508
345767
345940
345970
346059
346072
346133
346364
346501
346756
346816
346849
346891
346994
347015
347113
347250
347739
347811
347842
347872
347959
348041
348087
348114
348296
348320
348499
348599
348726
348781
348892
348956
348961
348977
349222
349304
349402
349468
349580
349594
349596
349661
349841
349902
349952
350147
350149
350164
350464
350569
350819
350937
351042
351232
351252
351256
351293
351317
351428
351640
351704
351816
352017
352159
352227
352243
352401
352561
352662
352781
352824
352941
352942
352960
353331
353484
353485
353500
353684
353703
353759
353762
353785
354086
354508
354735
355016
355047
355120
355165
355168
355253
355301
355544
355566
355602
355617
355904
355910
355933
356246
356253

512483
512660
512698
512702
512789
512965
513222
513418
513606
513829
514154
514456
514529
514534
514914
514987
515204
515337
515370
515426
515603
515754
515802
515958
516127
516154
516199
516260
516333
516467
516514
516543
516574
516629
516828
516915
516933
516938
517004
517157
517235
517271
517352
517453
517577
517732
517822
517839
517872
518098
518394
518410
518435
518574
518695
518705
519151
519255
519325
519464
519510
519520
519568
519718
519743
519769
519827
519898
519904
519916
520121
520330
520458
520462
520505
520566
520769
520806
520972
520986
521123
521170
521191
521250
521324
521366
521490
521513
521578
521624
521695
521841
522026
522134
522168
522185
522360
522379
522435
522498
522584
522711
522851
523008
523043
523079
523097
523147
523168
523182
523266
523296
523321
523341
523574
523684
523692
523722
523906
524058
524093
524393
525072
525585
525591
525622
525661
526106
526171
526215
526343
526382
526411
526456
526638
526874
526924
526955
526963
527171
527189
527362
527365

680124
680179
680259
680284
680392
680708
680783
681107
681589
681592
681783
681813
681892
681893
681939
681990
682022
682028
682114
682217
682244
682372
682445
682620
682699
682718
682969
683000
683114
683154
683155
683169
683205
683247
683336
683535
683579
683841
683846
683968
684011
684016
684068
684137
684148
684398
684410
684429
684431
684552
684643
684761
684777
684782
684784
684813
685007
685133
685220
685236
685272
685438
685488
685509
685640
685789
685853
685921
686153
686265
686274
686430
686684
686725
686793
687014
687016
687352
687466
687479
687509
687618
687720
687770
687816
687892
687899
688107
688339
688569
688575
688618
688770
689067
689093
689356
689480
689490
689591
689631
689670
689672
689723
689825
689879
689919
690162
690303
690425
690595
690818
690821
691034
691071
691470
691572
691624
691708
691731
691753
691781
691816
691889
692044
692741
693010
693033
693130
693472
693564
693589
693667
693738
693771
693778
693779
693939
693948
693963
694143
694263
694409
694475

794172
794188
794259
794374
794426
794571
794582
794739
795003
795107
795172
795256
795264
795272
795381
795542
795564
795819
795844
795851
796135
796137
796151
796256
796285
796316
796442
796508
796514
796623
796628
796689
796691
796875
797069
797209
797210
797338
797400
797414
797422
797748
797765
797809
797936
797948
798031
798147
798259
798282
798445
798492
798609
798721
799095
799278
799281
799344
799401
799578
799620
799626
799720
799827
799838
799887
799972
800042
800081
800523
800536
800539
800587
800687
800775
800834
800889
801175
801286
801451
801507
801524
801544
801655
801662
801688
801917
801921
802165
802293
802402
802442
802450
802593
802901
802902
803143
803284
803342
803412
803717
803722
803784
804117
804202
804213
804299
804304
804447
804600
804732
804753
804817
805030
805036
805127
805134
805194
805257
805325
805384
805462
805474
805478
805807
806062
806131
806185
806206
806371
806382
806727
806868
806926
807034
807063
807090
807169
807322
807447
807579
807637
807677

966279
966312
966496
966733
966923
967038
967118
967150
967376
967620
967775
967812
967870
967982
968185
968331
968332
968378
968473
968498
968591
968694
968821
968983
969089
969137
969140
969302
969394
969488
969553
969559
969734
969753
969976
970172
970340
970394
970405
970476
970734
970898
971658
971666
971670
971691
971712
971786
971819
971882
971928
971959
972005
972034
972107
972127
972233
972280
972310
972589
972607
972805
972854
972926
972972
973002
973043
973158
973243
973356
973427
973571
973641
973762
973848
973873
974064
974402
974530
974688
974766
975005
975177
975192
975251
975449
975475
975543
975582
975656
975795
975817
975883
975907
975917
975960
975963
976027
976117
976212
976218
976284
976406
976418
976456
976751
976793
976810
977132
977137
977264
977288
977483
977504
977585
977649
978071
978121
978139
978155
978197
978235
978245
978281
979173
979302
979341
979348
979353
979413
979551
979582
979735
979738
979764
979806
979946
979969
980153
980240
980465
980542
980640

In [17]:
#HeapSort:
def heapify(arr, n, i): 
    largest = i # Initialize largest as root 
    l = 2 * i + 1     # left = 2*i + 1 
    r = 2 * i + 2     # right = 2*i + 2 
  
    # See if left child of root exists and is 
    # greater than root 
    if l < n and arr[i] < arr[l]: 
        largest = l 
  
    # See if right child of root exists and is 
    # greater than root 
    if r < n and arr[largest] < arr[r]: 
        largest = r 
  
    # Change root, if needed 
    if largest != i: 
        arr[i],arr[largest] = arr[largest],arr[i] # swap 
  
        # Heapify the root. 
        heapify(arr, n, largest) 
  
# The main function to sort an array of given size 
def heapSort(arr): 
    n = len(arr) 
  
    # Build a maxheap. 
    for i in range(n, -1, -1): 
        heapify(arr, n, i) 
  
    # One by one extract elements 
    for i in range(n-1, 0, -1): 
        arr[i], arr[0] = arr[0], arr[i] # swap 
        heapify(arr, i, 0) 
  
# Driver code to test above 
arr = [ 12, 11, 13, 5, 6, 7] 
heapSort(long_list) 
n = len(long_list) 
print ("Sorted array is") 
for i in range(n): 
    print ("%d" %long_list[i]), 

Sorted array is
29
174
316
373
424
430
705
762
786
830
1017
1456
1652
1767
2115
2123
2241
2259
2450
2459
2765
2775
2776
2807
2973
3152
3183
3356
3376
3595
3678
3863
3892
3975
4016
4064
4090
4360
4772
4786
5280
5355
5875
5916
5919
5956
6060
6385
6493
6523
6599
6874
6916
6955
7093
7331
7332
7537
7669
8164
8266
8312
8388
8557
8604
8717
8860
9034
9274
9314
9430
9458
9521
9553
9588
9597
9808
9823
9891
9911
9944
9954
9990
10174
10204
10213
10252
10426
10757
10770
10945
10977
11019
11031
11380
11476
11507
11510
11658
11694
11757
11794
11869
11875
12006
12170
12191
12210
12344
12415
12430
12688
12711
12746
12752
12768
12819
12885
13022
13116
13135
13389
13457
13680
13809
13818
13822
13897
14208
14314
14506
14615
14626
14671
14885
14928
15079
15152
15172
15195
15317
15387
15415
15681
15796
16008
16051
16118
16196
16267
16332
16386
16450
16706
16889
16979
17002
17068
17344
17514
17556
17606
17632
17863
18069
18131
18220
18406
18479
18650
18787
18912
19120
19307
19326
19393
19411
19539
19652
1973

182942
182964
182970
183010
183046
183083
183112
183182
183331
183359
183370
183488
183527
183610
183814
183954
184197
184425
184589
184902
184907
185016
185202
185530
185655
185715
185870
185879
185895
185979
186110
186126
186147
186175
186316
186498
186508
186743
186950
187129
187188
187242
187388
187393
187475
187738
187771
187868
188078
188236
188270
188313
188452
188601
188654
188671
188680
188748
188839
188847
188988
188994
189003
189135
189247
189422
189680
189733
189949
190072
190096
190126
190212
190432
190515
190699
190827
190998
191017
191041
191086
191210
191337
191338
191588
191786
191854
191858
192078
192097
192177
192283
192541
192574
192597
192725
192764
192987
193114
193221
193247
193318
193412
193570
193697
193898
193933
194045
194106
194155
194362
194397
194732
194825
194976
194993
195023
195134
195328
195588
195642
195707
195929
196165
196434
196469
196482
196511
196574
196688
196872
196930
196948
196952
197072
197123
197241
197258
197712
197749
197756
197765
197780

337166
337199
337323
337360
337403
338089
338122
338378
338470
338555
338581
338799
338807
338849
339014
339158
339485
339491
339724
339780
339842
340309
340339
340359
340391
340432
340622
340878
341017
341036
341053
341098
341279
341286
341287
341320
341400
341450
341514
341734
342141
342153
342195
342355
342366
342370
342630
342662
342759
342911
343056
343069
343079
343115
343167
343233
343266
343300
343415
343546
343556
343841
344141
344298
344307
344321
344639
344711
344739
344748
344986
345030
345031
345168
345179
345299
345508
345767
345940
345970
346059
346072
346133
346364
346501
346756
346816
346849
346891
346994
347015
347113
347250
347739
347811
347842
347872
347959
348041
348087
348114
348296
348320
348499
348599
348726
348781
348892
348956
348961
348977
349222
349304
349402
349468
349580
349594
349596
349661
349841
349902
349952
350147
350149
350164
350464
350569
350819
350937
351042
351232
351252
351256
351293
351317
351428
351640
351704
351816
352017
352159
352227
352243

520330
520458
520462
520505
520566
520769
520806
520972
520986
521123
521170
521191
521250
521324
521366
521490
521513
521578
521624
521695
521841
522026
522134
522168
522185
522360
522379
522435
522498
522584
522711
522851
523008
523043
523079
523097
523147
523168
523182
523266
523296
523321
523341
523574
523684
523692
523722
523906
524058
524093
524393
525072
525585
525591
525622
525661
526106
526171
526215
526343
526382
526411
526456
526638
526874
526924
526955
526963
527171
527189
527362
527365
527574
527650
527723
527725
527828
527875
528062
528079
528290
528330
528332
528460
528537
528561
528904
528966
529101
529213
529345
529647
529653
529681
529766
529806
529994
530036
530062
530067
530083
530140
530176
530383
530505
530588
530619
530865
530908
530927
530993
531030
531100
531119
531218
531234
531298
531300
531486
531489
531746
531820
531854
531992
532198
532212
532229
532231
532263
532293
532334
532429
532625
532732
532990
533323
533394
533412
533484
533501
533600
533625
533642

689591
689631
689670
689672
689723
689825
689879
689919
690162
690303
690425
690595
690818
690821
691034
691071
691470
691572
691624
691708
691731
691753
691781
691816
691889
692044
692741
693010
693033
693130
693472
693564
693589
693667
693738
693771
693778
693779
693939
693948
693963
694143
694263
694409
694475
694629
694639
694674
694728
694777
694833
695009
695186
695202
695347
695408
695419
695435
695510
695607
695666
695720
695878
696049
696250
696442
696540
696570
696619
697012
697070
697467
697534
697556
697592
697596
697604
697634
697751
697753
697864
697955
698029
698043
698093
698124
698181
698202
698205
698293
698349
698350
698427
698479
698728
698788
698883
698909
698938
699117
699120
699158
699277
699311
699494
699611
699612
699649
699652
699714
699933
699956
700150
700247
700355
700406
700480
700889
700907
700937
701020
701192
701208
701239
701260
701278
701302
701412
701545
701575
701709
701794
701896
701974
701988
702006
702056
702205
702234
702624
702640
702641
702970

809249
809287
809309
809364
809569
809752
809814
809854
810196
810535
810553
810607
810721
810826
810839
810922
811074
811156
811319
811400
811422
811458
812039
812112
812130
812599
812661
812664
812866
813081
813407
813707
813738
814022
814057
814091
814129
814171
814203
814248
814257
814515
814851
814880
814894
815198
815298
815309
815358
815380
815399
815608
815701
815782
815880
816008
816105
816225
816231
816458
816476
816502
816553
816594
816640
816782
816830
816870
817013
817022
817088
817231
817241
817308
817525
817568
817616
817875
818072
818165
818214
818226
818318
818695
818932
819002
819011
819281
819310
819575
819635
819646
819742
819748
819867
820064
820065
820088
820098
820313
820581
820848
820976
821159
821202
821490
821559
821580
821599
821601
821614
821642
821844
821887
821941
821996
822078
822109
822361
822436
822731
822910
822915
822970
823165
823222
823418
823510
823706
823717
823792
823803
823869
824107
824182
824837
824859
824874
824938
825086
825113
825406
825410

In [20]:
#QuickSort:
def partition(arr,low,high): 
    i = ( low-1 )         # index of smaller element 
    pivot = arr[high]     # pivot 
  
    for j in range(low , high): 
  
        # If current element is smaller than or 
        # equal to pivot 
        if   arr[j] <= pivot: 
          
            # increment index of smaller element 
            i = i+1 
            arr[i],arr[j] = arr[j],arr[i] 
  
    arr[i+1],arr[high] = arr[high],arr[i+1] 
    return ( i+1 ) 
  
# The main function that implements QuickSort 
# arr[] --> Array to be sorted, 
# low  --> Starting index, 
# high  --> Ending index 
  
# Function to do Quick sort 
def quickSort(arr,low,high): 
    if low < high: 
  
        # pi is partitioning index, arr[p] is now 
        # at right place 
        pi = partition(arr,low,high) 
  
        # Separately sort elements before 
        # partition and after partition 
        quickSort(arr, low, pi-1) 
        quickSort(arr, pi+1, high)
        
# Driver code to test above 
arr = [10, 7, 8, 9, 1, 5] 
n = len(long_list) 
quickSort(long_list,0,n-1) 
print ("Sorted array is:") 
for i in range(n): 
    print ("%d" %long_list[i]),



RecursionError: maximum recursion depth exceeded in comparison